# GLUsyn testing

This is a test suite for GABA-ergic synapses on the Gillies-Willshaw STN neuron model, using GABAsyn.mod

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook
# Enable connecting with ipyton console --existing
%connect_info
# Enabling printing stdout and stderr not captured by notebook
from IPython.utils import io # with io.capture_output() as captured: print captured.stdout

In [ ]:
# NEURON modules
import neuron
from neuron import h
h.load_file("stdlib.hoc") # Load the standard library
h.load_file("stdrun.hoc") # Load the standard run library

# Add our own modules to Python path
import sys, os.path
scriptdir = os.path.abspath('.')
modulesbase = os.path.normpath(os.path.join(scriptdir, '..'))
sys.path.append(modulesbase)

# Gillies-Willshaw STN model
import gillies_model as gillies

# Plotting & recording
from bgcellmodels.common import analysis

# Physiological parameters
import bgcellmodels.cellpopdata as cpd
from bgcellmodels.cellpopdata import PhysioState, Populations as Pop, NTReceptors as NTR, ParameterSource as Cit

# STN cell testing
from stn_model_evaluation import *

## Configure synapse

First we define a function to set up the inputs. This is an extension function to `stn_model_evaluation.py/STNModelEvaluator`

In [ ]:
def make_GLU_inputs(self, n_ctx_syn=1):
    """
    Make a single Glutamergic synapse on the STN neuron.
    """
    cc = cpd.CellConnector(self.physio_state, self.rng)
        
    # Add CTX inputs using Tsodyks-Markram synapses
    ctx_syns = []
    ctx_ncs = []
    ctx_stims = []
    ctx_hoc_handlers = []
    ctx_py_handlers = []
    ctx_wvecs = []

    # Distribute synapses over dendritic trees
    is_ctx_target = lambda seg: seg.diam <= 1.0			
    dendrites = self.model_data[self.target_model]['sec_refs']['dendrites']
    dend_secrefs = sum(dendrites, [])
    ctx_target_segs = pick_random_segments(dend_secrefs, n_ctx_syn, is_ctx_target, rng=self.rng)

    # Make synapses
    for target_seg in ctx_target_segs:

        # Make poisson spike generator
        stim_rate = 20.0 # hz
        stim_T = stim_rate**-1*1e3
        stimsource = h.NetStim() # Create a NetStim
        stimsource.interval = stim_T # Interval between spikes
        stimsource.number = 1e9 # max number of spikes
        stimsource.noise = 0.0 # Fractional noise in timing
        ctx_stims.append(stimsource) # Save this NetStim

        # Custom synapse parameters
        syn_mech = 'GLUsyn'
        syn_params = {
            'U1': 0.7,
            'tau_rec': 200., # 1./20. / 2. * 1e3, # 95% recovery of RRP under 20Hz stim (Gradinaru 2009)
            'tau_facil': 1., # no facilitation
        }

        # Make synapse and NetCon
        syn, nc, wvecs = cc.make_synapse((Pop.CTX, Pop.STN), (stimsource, target_seg), 
                            syn_mech, (NTR.AMPA, NTR.NMDA), (Cit.Custom, Cit.Default),
                            custom_synpar=syn_params)

        print("Made {} synapse with following parameters:".format(syn_mech))
        for pname in cc.getSynMechParamNames(syn_mech):
            print("{} : {}".format(pname, str(getattr(syn, pname))))

        # Compensate for effect max value Hill factor and U1 on gmax_GABAA and gmax_GABAB
        syn.gmax_AMPA = syn.gmax_AMPA / syn.U1
        syn.gmax_NMDA = syn.gmax_NMDA / syn.U1

        # Control netstim
        tstart, tstop = 700, 1300
        stimsource.start = tstart
        turn_off = h.NetCon(None, stimsource)
        turn_off.weight[0] = -1
        def queue_events():
            turn_off.event(tstop)
        fih = h.FInitializeHandler(queue_events)
        
        ctx_py_handlers.append(queue_events)
        ctx_hoc_handlers.append(fih)
        ctx_ncs.append(nc)
        ctx_ncs.append(turn_off)
        ctx_syns.append(syn)
        ctx_wvecs.extend(wvecs)

    # Save inputs
    new_inputs = {
        'stimweightvec': ctx_wvecs,
        'synapses': ctx_syns,
        'NetCons': ctx_ncs,
        'NetStims': ctx_stims,
        'PyInitHandlers': ctx_py_handlers,
        'HocInitHandlers': ctx_hoc_handlers,
    }
    self.add_inputs('ctx', **new_inputs)

## Configure recording

Record variables of interest

In [ ]:
def rec_traces(self, protocol, recordStep=0.025):
    """
    Set up recording Vectors
    """
    somasec = h.SThcell[0].soma
    dendsec = h.SThcell[0].dend1[7]

    # Assign label to each recorded section
    rec_segs = {
        'soma': somasec(0.5), # middle of soma
        'dist_dend': dendsec(0.8), # approximate location along dendrite in fig. 5C
    }

    self.model_data[self.target_model]['rec_segs'] = rec_segs
    
    # END COMMON PART ##############################################################
    
    # Start trace specification
    traceSpecs = collections.OrderedDict() # for ordered plotting (Order from large to small)
    traceSpecs['t_global'] = {'var':'t'}
    self.rec_dt = recordStep
    
    # Add synapse and segment containing it
    nc = self.model_data[self.target_model]['inputs']['ctx']['NetCons'][0]
    rec_segs['synGLU'] = nc.syn()
    rec_segs['postseg'] = nc.syn().get_segment()

    # Record synaptic variables
    traceSpecs['gA_syn'] = {'pointp':'synGLU', 'var':'g_AMPA'}
    traceSpecs['gN_syn'] = {'pointp':'synGLU', 'var':'g_NMDA'}
    traceSpecs['Rrp_syn'] = {'pointp':'synGLU', 'var':'R'}
    traceSpecs['Use_syn'] = {'pointp':'synGLU', 'var':'Use'}

    # Record membrane voltages
    for seclabel, seg in rec_segs.iteritems():
        if isinstance(seg, neuron.nrn.Segment):
            traceSpecs['V_'+seclabel] = {'sec':seclabel, 'loc':seg.x, 'var':'v'}
            
    # Prepare dictionary (label -> Section)
    rec_secs = {}
    for seclabel, hobj in rec_segs.iteritems():
        if isinstance(hobj, neuron.nrn.Segment):
            rec_secs[seclabel] = hobj.sec
        else:
            rec_secs[seclabel] = hobj # point process

    # Use trace specs to make Hoc Vectors
    recData = analysis.recordTraces(rec_secs, traceSpecs, recordStep)

    # Save trace specs and recording Vectors
    self.model_data[self.target_model]['rec_data'][protocol] = {
        'trace_specs': traceSpecs,
        'trace_data': recData,
        'rec_dt': recordStep,
    }

## Configure ploting
Plot variables of interest

In [ ]:
def plot_traces(self, protocol):
    """
    Plot traces for our simulation
    """
    # Get data
    model = self.target_model
    traceSpecs = self.model_data[model]['rec_data'][protocol]['trace_specs']
    recData = self.model_data[model]['rec_data'][protocol]['trace_data']
    recordStep = self.model_data[model]['rec_data'][protocol]['rec_dt']
    
    # END COMMON PART ##############################################################
    
    # Plot membrane voltages (one figure)
    recV = analysis.match_traces(recData, lambda t: t.startswith('V_'))
    figs_vm = analysis.plotTraces(recV, recordStep, yRange=(-80,40), 
                                    traceSharex=True, oneFigPer='cell')

    # Plot synaptic variables
    syn_traces = analysis.match_traces(recData, lambda t: (not t.startswith('V_')) and t.endswith('syn'))
    analysis.plotTraces(syn_traces, recordStep, traceSharex=True, title='Synaptic variables')

## Run simulation

Now we simulate the cell with this input.

In [ ]:
evaluator = StnModelEvaluator(StnModel.Gillies2005, PhysioState.NORMAL)
evaluator.build_cell(StnModel.Gillies2005)
proto = StimProtocol.SINGLE_SYN_GLU

# This sequence is equivalent to simulate_protocol()
evaluator.init_sim()
make_GLU_inputs(evaluator)

# Show what we are simulating
with io.capture_output() as captured:
    h.topology()
    print captured.stdout
    print captured.stderr

for syn in evaluator.model_data[evaluator.target_model]['inputs']['ctx']['synapses']:
    print("Synapse {} in segment {}".format(syn, syn.get_segment()))

In [ ]:
rec_traces(evaluator, proto)
evaluator.run_sim()
plot_traces(evaluator, proto) 